<a href="https://colab.research.google.com/github/tanishavaishya18/python-basics/blob/main/Day13_MiniProject_EndToEnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROBLEM STATEMENT**

Predict wine class quality using chemical properties, and build a robust ML pipeline with proper evaluation

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

data = load_wine()
x=data.data
y=data.target
feature_names=data.feature_names

df = pd.DataFrame(x, columns=feature_names)
df['class']=y
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [4]:
df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [3]:


df.isnull().sum()

,0
alcohol,0
malic_acid,0
ash,0
alcalinity_of_ash,0
magnesium,0
total_phenols,0
flavanoids,0
nonflavanoid_phenols,0
proanthocyanins,0
color_intensity,0


In [5]:

df['acid_ratio']=df['malic_acid']/(df['total_phenols']+1e-6)
df['flav_color_interaction']=df['flavanoids']*df['color_intensity']
df['alcohol_proline_ratio']=df['alcohol']/(df['proline']+ 1e-6)

In [6]:
x=df.drop('class', axis =1)
y=df['class']

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42))
])

In [11]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipe, x, y, cv=5, scoring='accuracy')
print('CV scores', scores)
print('mean accuracy', scores.mean())
print('STD Dev', scores.std())

CV scores [0.97222222 0.94444444 0.94444444 0.97142857 0.97142857]
mean accuracy 0.9607936507936508
STD Dev 0.013352216409405144


In [16]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'model__n_estimators' : [50, 100, 200],
    'model__max_depth' : [None, 5, 10]
    }

grid = GridSearchCV(
    pipe, param_grid, cv=5, scoring='accuracy'
)
grid.fit(x,y)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'model__max_depth': [None, 5, 10],
                         'model__n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [18]:
grid.best_params_


{'model__max_depth': 5, 'model__n_estimators': 50}

In [19]:
grid.best_score_

np.float64(0.9665079365079364)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

best_model = grid.best_estimator_
best_model.fit(x_train, y_train)

pred = best_model.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [22]:
#Reflection

**Why Pipleines prevent leakage**

Pipelines ensure that preprocessing steps like scaling and feature transformation are learned only from the training data and then applied to the test data, preventing information from leaking and inflating performance metrics.

**Why cross validation is important**

Cross-validation provides a more reliable estimate of model performance by evaluating it across multiple train-test splits, reducing dependence on a single random split and improving result stability.

**How feature emgineering helped**

Feature engineering created more informative representations of the data by capturing ratios and interactions between variables, which improved class separability and helped the model learn more meaningful patterns.

**WHy Random Forest worked well**

Random Forest performed well because it combines multiple decision trees, reducing overfitting while capturing non-linear relationships and interactions present in the dataset.

**How this approach generalises real data**

This workflow—feature engineering, pipelines, cross-validation, and tuning—is robust to noisy and complex real-world datasets, making it suitable for applications such as battery health estimation, renewable energy mapping, and sensor-based systems.